In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import sys
from ipywidgets import widgets
from IPython.display import display, HTML
from IPython.display import clear_output
import matplotlib.pyplot as plt
import seaborn as sns

# Customized modules
HOME = '/home/serinatan/project/GPU-Virtualization-Benchmarks/util'
if HOME not in sys.path:
    sys.path.append(HOME)

import data.scripts.common.format as fmt
import data.scripts.common.constants as const
import data.scripts.gen_tables.gen_pair_configs as gen_pair

SCRIPT_PATH = os.path.join(HOME, 'data/scripts') 
PKL_PATH = os.path.join(HOME, 'data/pickles') 

In [3]:
def draw_table(df, cols, hide_index=True):
    if hide_index:
        return df[cols].style.set_table_styles(fmt.table_style).hide_index()
    else:
        return df[cols].style.set_table_styles(fmt.table_style)

# Seq

In [10]:
# run python scripts to generate all the pickles needed
gen_seq = os.path.join(SCRIPT_PATH, 'gen_tables/gen_table_seq.py')   
%run $gen_seq 

df_seq = pd.read_pickle(os.path.join(PKL_PATH, 'seq.pkl'))
print(df_seq.columns)

Index(['pair_str', 'config', 'gpusim_version', 'jobId', 'grid_x', 'grid_y',
       'grid_z', 'block_x', 'block_y', 'block_z', 'ctas/SM', 'runtime', 'ipc',
       'instructions', 'avg_mem_lat', 'avg_core_to_l2', 'avg_l2_to_core',
       'avg_mrq_latency', 'stall_core_ldst', 'stall_icnt_to_l2', 'l2_BW',
       'l2_rshr_entry_fail', 'l2_rshr_merge_fail', 'stall_l2_to_icnt',
       'l1D_miss_rate', 'l2_miss_rate', 'l2_total_accesses', 'packet_lat_out',
       'network_lat_out', 'inject_out', 'accepted_out', 'packet_lat_in',
       'network_lat_in', 'inject_in', 'accepted_in',
       'mem_subpartition_parallism', 'mem_subpartition_parallism_util',
       'L2_reservation_fail', 'empty_warp', 'stall_warp', 'idle_warp',
       'scoreboard_warp', 'tot_warp_insn', 'regs', 'smem', 'dram_eff',
       'dram_bw', 'row_buffer_locality', 'mrqq', 'total_cmd', 'wasted_col',
       'wasted_row', 'mem_idle', 'CCDLc', 'WTRc', 'RTWc', 'RCDc', 'RCDWRc',
       'avg_dram_bw', 'avg_dram_eff', 'avg_row_locality

In [13]:
col_seq = ['pair_str', 'config', 'runtime', 'instructions', 'ipc', 'avg_dram_bw', 
           'ratio_dram_bw', 'avg_dram_eff', 'avg_mem_lat', 'MPKI', 'waves',
           'avg_row_locality',
#            'l2_total_accesses', 'l2_access_density',
          ]
df_seq.sort_values('avg_row_locality', inplace=True)
draw_table(df_seq, col_seq).format(
    {'avg_dram_bw':'{:.4f}', 
     'ratio_dram_bw': '{:.2f}',
     'MPKI': '{:.2f}',
     'waves': '{:.2f}',
     'l2_total_accesses': '{:,}',
     'l2_access_density': '{:.1f}',
     'instructions': '{:,}',
    })

pair_str,config,runtime,instructions,ipc,avg_dram_bw,ratio_dram_bw,avg_dram_eff,avg_mem_lat,MPKI,waves,avg_row_locality
parb_lbm-0,TITANV-SEP_RW,875934,"427,778,603",488.367,0.7181,0.05,0.742696,14605,26.30,18.75,0.0499595
parb_stencil-0,TITANV-SEP_RW,350242,"644,994,210",1841.56,0.7136,0.01,0.7437,1229,6.68,0.80,0.0752099
cut_wmma-1,TITANV-SEP_RW,2257808,"2,045,247,488",905.855,0.5092,0.01,0.706908,3280,9.58,3.20,0.095261
rod_hotspot3d-0,TITANV-SEP_RW,60893,"83,099,648",1364.64,0.7073,0.00,0.782258,815,9.48,1.60,0.122337
parb_spmv-0,TITANV-SEP_RW,73206,"52,720,795",720.151,0.7725,0.00,0.859517,795,18.51,0.90,0.164629
rod_streamcluster-0,TITANV-SEP_RW,159207,"133,834,562",840.622,0.8230,0.00,0.926196,570,16.64,0.53,0.183334
rod_lavamd-0,TITANV-SEP_RW,4664594,"13,474,724,576",2888.72,0.0114,0.01,0.168133,1329,0.09,1.39,0.240969
cut_sgemm-1,TITANV-SEP_RW,1719066,"6,580,076,544",3827.7,0.0742,0.00,0.465546,931,0.34,3.20,0.387708
cut_sgemm-0,TITANV-SEP_RW,93149,"228,392,960",2451.86,0.0883,0.00,0.570446,298,1.26,0.80,0.388526
rod_pathfinder-0,TITANV-SEP_RW,33973,"126,061,000",3710.4,0.4545,0.00,0.585313,319,2.21,0.72,0.401707


# Intra

In [18]:
gen_intra = os.path.join(SCRIPT_PATH, 'gen_tables/gen_table_intra.py')
%run $gen_intra

In [19]:
df_intra = pd.read_pickle(os.path.join(PKL_PATH, 'intra.pkl'))
col_intra = ['pair_str', 'intra', 'l2', 'norm_ipc']

# draw_table(df_intra, col_intra)

In [20]:
df_intra_best = pd.read_pickle(os.path.join(PKL_PATH, 'intra_best.pkl'))
df_intra_best.sort_values('pair_str', inplace=True)
col_intra_best = ['pair_str', 
        'perfdollar', 'intra', 'l2', 'norm_ipc', 
        'runtime',
        #'l2_miss_rate',
        #'avg_mem_lat', 'avg_core_to_l2', 'avg_l2_to_core',
        'comp_busy', 'dram_busy', 'avg_dram_bw', 'avg_dram_eff']

draw_table(df_intra_best, col_intra_best).format({'norm_ipc': "{:.4f}", 'dominant_resc': '{:.2f}'})

pair_str,perfdollar,intra,l2,norm_ipc,runtime,comp_busy,dram_busy,avg_dram_bw,avg_dram_eff
cut_sgemm-0,0.669156,2,0.5,0.9614,96892,0.259849,0.181625,0.129729,0.644896
cut_sgemm-1,1.43829,1,0.5,1.1162,1540155,0.460856,0.205192,0.113583,0.428508
cut_wmma-0,1.68589,2,0.5,0.9463,66533,0.0559222,0.0935402,0.0432762,0.299054
cut_wmma-1,0.886275,2,0.5,0.9722,2322385,0.0999374,0.733181,0.501917,0.667796
parb_cutcp-0,3.27202,2,0.5,1.0000,525176,0.127457,9.28537e-05,2.21786e-05,0.0990083
parb_lbm-0,0.700971,3,0.5,0.9121,960397,0.0511992,0.980072,0.655588,0.667817
parb_spmv-0,0.725113,6,0.5,1.0264,71326,0.0759997,0.927519,0.814725,0.872437
parb_stencil-0,0.726218,3,0.5,0.9429,371444,0.221282,0.959338,0.6521,0.671542
rod_heartwall-0,3.17514,1,0.5,1.0000,10070,0.0646486,0.00632534,0.00137254,0.09346
rod_hotspot-0,2.36684,2,0.5,0.9012,32531,0.329445,0.118075,0.0683838,0.322858


# Algorithm: Find pair configs

In [17]:
col_prod = ['norm_ipc_x', 'norm_ipc_y', 'diff_mflat', 'sum_ipc', 
            'intra_x', 'intra_y', 'l2_x', 'l2_y',
           'sum_comp', 'sum_dram', 'penalized']
find_pair = os.path.join(SCRIPT_PATH, 'gen_tables/gen_pair_configs.py')


app1 = widgets.Dropdown(
    options=df_seq['pair_str'],
    value='cut_sgemm-0',
    rows=5,
    description='App 1:',
    disabled=False
)

app2 = widgets.Dropdown(
    options=df_seq['pair_str'],
    value='cut_sgemm-0',
    rows=5,
    description='App 2:',
    disabled=False
)

qos = widgets.FloatSlider(
    value=0.75,
    min=0.1,
    max=0.95,
    step=0.05,
    description='QoS:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

button = widgets.Button(description='Calculate', button_style='info')

def show_widgets():
    display(app1)
    display(app2)
    display(qos)
    display(button)
    
show_widgets()

def onclick(b):
    clear_output()
    show_widgets()
    %run $find_pair --app $app1.value $app2.value --qos $qos.value
    
    df_prod = pd.read_pickle(os.path.join(PKL_PATH, 'pair_candidates.pkl'))
    print(df_prod.columns)
#     sns.lineplot('diff_mflat', 'sum_ipc', data=df_prod)
   
    display(HTML(draw_table(df_prod, col_prod, False).render()))
    
    
button.on_click(onclick)

Dropdown(description='App 1:', index=4, options=('parb_lbm-0', 'parb_stencil-0', 'cut_wmma-1', 'rod_hotspot3d-…

Dropdown(description='App 2:', index=10, options=('parb_lbm-0', 'parb_stencil-0', 'cut_wmma-1', 'rod_hotspot3d…

FloatSlider(value=0.75, continuous_update=False, description='QoS:', max=0.95, min=0.1, step=0.05)

Button(button_style='info', description='Calculate', style=ButtonStyle())

Index(['pair_str_x', 'config_x', 'gpusim_version_x', 'jobId_x', 'grid_x_x',
       'grid_y_x', 'grid_z_x', 'block_x_x', 'block_y_x', 'block_z_x',
       ...
       'thread_ratio_y', 'smem_ratio_y', 'reg_ratio_y', 'usage_y',
       'diff_mflat', 'sum_ipc', 'sum_comp', 'sum_dram', 'penalized', 'config'],
      dtype='object', length=153)


,norm_ipc_x,norm_ipc_y,diff_mflat,sum_ipc,intra_x,intra_y,l2_x,l2_y,sum_comp,sum_dram,penalized
239,0.976302,1,13,1.9763,6,8,0.25,0.75,0.470041,0.931962,parb_spmv-0
171,1.02636,1.01901,24,2.04537,6,6,0.5,0.5,0.466431,1.03083,rod_hotspot-0
199,0.755625,0.981193,26,1.73682,3,4,0.75,0.25,0.422565,1.19799,parb_spmv-0
126,0.750882,1.00089,29,1.75177,3,4,0.5,0.5,0.43408,1.04297,rod_hotspot-0
50,1.02254,0.93776,30,1.9603,9,8,0.75,0.25,0.456639,1.13091,rod_hotspot-0
140,1.04622,0.964442,53,2.01066,6,6,0.75,0.25,0.451618,1.18846,parb_spmv-0
198,0.755625,0.873347,62,1.62897,3,2,0.75,0.25,0.371527,1.1772,rod_hotspot-0
234,0.976302,1.02949,75,2.0058,6,6,0.25,0.75,0.469182,0.931974,rod_hotspot-0
168,1.02636,0.995349,78,2.02171,6,8,0.5,0.5,0.468456,1.02229,parb_spmv-0
135,1.04622,0.981193,81,2.02741,6,4,0.75,0.25,0.444981,1.19614,rod_hotspot-0


In [94]:
list_pairs = []
list_app_pairs = []
qos = 0.75

for bench1 in const.app_for_pair:
    for bench2 in const.app_for_pair:
        if bench1 < bench2:
            list_app_pairs.append([bench1, bench2])
#             config_str = gen_pair.main(['--app', bench1, bench2])
#             if len(config_str) > 0: 
#                 df_first = pd.read_pickle(os.path.join(PKL_PATH, 'pair_candidates.pkl'))
#                 list_pairs.append(df_first.head(1).reset_index(drop=True))
            

# df_pairs = pd.concat(list_pairs, axis=0).reset_index()
# df_pairs.sort_values(['sum_dram', 'sum_comp'], ascending=False, inplace=True)
# draw_table(df_pairs, ['pair_str_x', 'pair_str_y'] + col_prod, False)

for idx, p in enumerate(list_app_pairs):
      print(idx, p)

0 ['cut_sgemm-1', 'cut_wmma-0']
1 ['cut_sgemm-1', 'parb_cutcp-0']
2 ['cut_sgemm-1', 'parb_stencil-0']
3 ['cut_sgemm-1', 'parb_lbm-0']
4 ['cut_sgemm-1', 'parb_spmv-0']
5 ['cut_sgemm-1', 'rod_heartwall-0']
6 ['cut_sgemm-1', 'rod_hotspot-0']
7 ['cut_sgemm-1', 'rod_hotspot3d-0']
8 ['cut_sgemm-1', 'rod_streamcluster-0']
9 ['cut_sgemm-1', 'rod_pathfinder-0']
10 ['cut_sgemm-1', 'rod_lavamd-0']
11 ['cut_wmma-0', 'parb_cutcp-0']
12 ['cut_wmma-0', 'parb_stencil-0']
13 ['cut_wmma-0', 'parb_lbm-0']
14 ['cut_wmma-0', 'parb_spmv-0']
15 ['cut_wmma-0', 'rod_heartwall-0']
16 ['cut_wmma-0', 'rod_hotspot-0']
17 ['cut_wmma-0', 'rod_hotspot3d-0']
18 ['cut_wmma-0', 'rod_streamcluster-0']
19 ['cut_wmma-0', 'rod_pathfinder-0']
20 ['cut_wmma-0', 'rod_lavamd-0']
21 ['parb_cutcp-0', 'parb_stencil-0']
22 ['parb_cutcp-0', 'parb_lbm-0']
23 ['parb_cutcp-0', 'parb_spmv-0']
24 ['parb_cutcp-0', 'rod_heartwall-0']
25 ['parb_cutcp-0', 'rod_hotspot-0']
26 ['parb_cutcp-0', 'rod_hotspot3d-0']
27 ['parb_cutcp-0', 'rod_stream